In [0]:
%tensorflow_version 1.x
import numpy as np
import keras
import tensorflow as tf
from keras.layers import Dense, Conv1D, SpatialDropout1D, MaxPooling1D,Input, Flatten
from keras.layers.core import Lambda, Dropout
from keras import backend as K
from keras.regularizers import l2
from keras.models import Model
import os

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Loading 5' splicing data


nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1
def prob_SD1 (sd1_freq, sd2_freq):
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr)))
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1
    return arr_rep

#Storing model inputs and outputs
dataset_path = os.path.expanduser("/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/5SS_compressed.txt")
seq_len = 101
n = 265137
inputs = np.zeros((n,seq_len, 4))
prob_s1 = np.zeros(n)
with open(dataset_path) as f:
    ind = 0
    for line in f:
        mod_line = line.split('\t')
        inputs[ind] = seq_to_arr(mod_line[1])
        prob_s1[ind] = prob_SD1(float(mod_line[2]), float(mod_line[3][:-1]))
        ind += 1

#Loading Data
X = np.array(inputs)
y = np.array(prob_s1)

In [0]:

def create_dropout_masks(dim, drop_prob):
    """Creates dropout masks that are fixed across all model inputs for use in estimation of Var(Y_{sample})
    Note: this function is specific to keras models used in this experiment; number of dropout masks J is deduced from dim.
    #Arguments
      dim: the numpy array shape of values entering the generated dropout masks for all points across all J fixed dropout masks
      drop_prob: the dropout probability
    #Returns
      dropout masks used in estimation of Var(Y_{sample})
    """
    if (len(dim)==4):
        return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3])), p=[drop_prob, 1-drop_prob])
    return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3], dim[4])), p=[drop_prob, 1-drop_prob])

def apply_dropout_masks(a,b):
    return np.squeeze(np.multiply(a,b))

def multi_mask_predict(layer_fn, multi_mask_input):
    """Applies Keras model layers to multiple arrays of layer inputs for all points across J fixed dropout masks
    #Arguments
        layer_fn: keras backend function applying the mapping corresponding to keras model layers
        multi_mask_input: numpy array of layer inputs for all points across J fixed dropout masks
    #Returns
        predictions from layer_fn applied to multi_mask_input as a numpy array
    """
    layer_output = []
    for mask_num in range(multi_mask_input.shape[0]):
        layer_output.append(layer_fn((multi_mask_input[mask_num], 1)))
    return np.array(layer_output)


#Used for calculating test MSE (average of MC dropout predictions)
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean and variance of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
      f: function mapping model input and Keras backend learning_phase flag to model output
      x: input
      n_iter: number of repreated MC dropout predictions per point
    #Returns:
      Mean and variance of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])[:,0]
        result[i,:] = predictions
    prediction = result.mean(axis=0)
    uncertainty = result.var(axis=0)
    return prediction, uncertainty

In [0]:

def fixed_mask_forward_pass(model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks):
      """Makes model predictions with J dropout masks that are fixed across points to enable estimation of Var(Y_{sample})
    Function is specific to the given Keras model.
    #Arguments
        model: keras model
        forward_pass_input: X_{sample}
        num_masks: J, the number of dropout masks being used in estimation of Var(Y_{sample}) and calculation of the EI acquisition function
        dropout_prob: dropout probability 
        conv_masks: first set of dropout masks, applied after second MaxPooling1D layer
        dense_masks: second set of dropout masks, applied after the first Dense layer
    #Returns
        MC dropout predictions across sample points enabling estimation of Var(Y_{sample}), i.e. \hat{Y}_{sample}
    """
    # Functions to retrieve output of intermediate layers
    # Needed for manual implementation of fixed dropout masks 
    # across all data points
    conv = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[4].output])

    dense_1 = K.function([model.layers[6].input, K.learning_phase()],
                   [model.layers[7].output])

    dense_2 = K.function([model.layers[9].input, K.learning_phase()],
                   [model.layers[9].output])
    

    conv_output = np.array(conv((forward_pass_input, 1)))
    dense_1_input = apply_dropout_masks(conv_output, conv_masks)
    dense_1_output = multi_mask_predict(dense_1, dense_1_input)
    dense_2_input = apply_dropout_masks(dense_1_output, dense_masks)
    dense_2_output = np.squeeze(multi_mask_predict(dense_2, dense_2_input))
    return dense_2_output
  

def ei_acquisition_fn_model_var (univ_covariance, num_pool_samples, num_training_samples, batch_size):
    """Given Var(Y_{sample}), applies batch-mode EI active learning to query points
    #Arguments
        univ_covariance: Var(Y_{sample})
        num_pool_samples: number of pool points in D_{sample}
        num_training_samples: number of training points in D_{sample}
        batch_size: number of queried points per batch
    #Returns
        the indices of queried pool points as they are arranged in univ_covariance
    """
    acq_ind = []
    for acq_num in range(batch_size):
        all_acq_values = np.zeros(num_pool_samples)
        for new_pt_ind in range(num_pool_samples):
            covariance_vector = univ_covariance[num_training_samples+new_pt_ind,:]
            all_acq_values[new_pt_ind] = np.sum(np.square(covariance_vector))/(univ_covariance[num_training_samples+new_pt_ind, num_training_samples+new_pt_ind])
        sorted_top_ind = np.flip(np.argsort(all_acq_values))
        found_new_ind = False
        top_ind_ctr = -1
        while (found_new_ind == False):
            top_ind_ctr += 1
            new_top_ind = sorted_top_ind[top_ind_ctr]
            if new_top_ind not in acq_ind:
                acq_ind.append(new_top_ind)
                found_new_ind = True
        top_cov_vector = np.expand_dims(univ_covariance[num_training_samples+acq_ind[-1], :], axis=1)
        univ_covariance = univ_covariance - np.matmul(top_cov_vector, top_cov_vector.T)/univ_covariance[num_training_samples+acq_ind[-1], num_training_samples+acq_ind[-1]]
    return acq_ind

  
def get_acquisition_fn (model, X_train_sample, X_cand, num_masks, tau_inverse, batch_size, dropout_prob):
    """Given sample points, generates J fixed-mask predictions across all sample points, calculates Var(Y_{sample}), and runs batch-mode EI acquisition
    #Arguments
      model: keras model
      X_train_sample: numpy array of sample points that are in the training set
      X_cand: numpy array of sample points that are in the pool
      num_masks: J, the number of dropout masks being used in estimation of Var(Y_{sample}) and calculation of the EI acquisition function
      tau_inverse: inverse model precision hyperparameter
      batch_size: number of queried points per batch
      dropout_prob: dropout probability 
    #Returns
      The indices of pool points queried by batch-mode EI as they are arranged in X_{cand}
    """
    forward_pass_input = np.concatenate((X_train_sample, X_cand))
    forward_pass_output = np.zeros((len(X_train_sample)+len(X_cand), num_masks))
    #Generating fixed dropout masks
    conv_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 9 , 50)), p=[dropout_prob, 1-dropout_prob])
    dense_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 1 , 50)), p=[dropout_prob, 1-dropout_prob])
    #Generating fixed mask predictions in 2000 point chunks (to avoid exceeding memory limits)
    last_point_ind = 0
    while last_point_ind < len(X_train_sample) + len(X_cand):
      if last_point_ind+2000<len(X_train_sample) + len(X_cand):
        forward_pass_chunk = forward_pass_input[last_point_ind:last_point_ind+2000]
        forward_pass_output[last_point_ind:last_point_ind+2000] = fixed_mask_forward_pass(model, forward_pass_chunk, num_masks, dropout_prob, conv_masks, dense_masks).T
      else:
        forward_pass_chunk = forward_pass_input[last_point_ind:]
        forward_pass_output[last_point_ind:] = fixed_mask_forward_pass(model, forward_pass_chunk, num_masks, dropout_prob, conv_masks, dense_masks).T
      last_point_ind += 2000
    output_covariance = np.cov(forward_pass_output)
    print('num training: ' + str(len(X_train_sample))+', num cand: ' + str(len(X_cand)))
    print('Tau Inverse Value: ' + str(tau_inverse))
    #Var(Y_{sample}) = Var(\hat{Y}_{sample}) + tau^{-1} I
    final_output_covariance = output_covariance + (tau_inverse * np.identity(output_covariance.shape[0]))
    return ei_acquisition_fn_model_var(final_output_covariance, len(X_cand), len(X_train_sample), batch_size) 



In [0]:
#Active Learning Experiment Parameters
num_acquisitions = 1000
batch_size = 1
sample_size=5000
acq_fn_dropout_iterations = 50
mse_dropout_iterations = 200
num_epochs = 300
size_train = 75
#Training batch size (not active learning batch size)
batchsize=128
tau_inverse = 0.00025 
dropout_prob = 0.15
#This is run for three values of ind: 1, 2 and 3
ind = 1



In [0]:
def initialize_model(curr_size_train):
  """Used to initialize a Keras model in each active learning iteration; enables model retraining from scratch in each iteration
  #Arguments:
    curr_size_train: current training size, which affects the weight decay
  #Returns:
    A new initialized Keras model
  """
  #Parameters
  dropout_prob = 0.15
  Weight_Decay = 0.025/curr_size_train
  #Model
  inputlayer = Input(shape=(seq_len, len(nuc_arr)))
  x = Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu')(inputlayer)
  x = MaxPooling1D(pool_size=3)(x)
  x = Conv1D(seq_len//2, (4), strides=1, activation='relu')(x)
  x = MaxPooling1D(pool_size=3)(x)
  x = Dropout(dropout_prob)(x)
  x = Flatten()(x)
  x = Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu')(x)
  x = Dropout(dropout_prob)(x)
  outputlayer = Dense(1, W_regularizer=l2(Weight_Decay), init='normal')(x)

  model = Model(inputlayer, outputlayer)
  model.compile(loss='mean_squared_error', optimizer='rmsprop')
  return model


In [0]:
#Initializing datasets and filepaths
#Note: code is currently set to *resume* active learning experiments. 
#To begin active learning experiments, several commented lines below should be uncommented and vice versa.

filepath_ei_mse = '/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Results_v3/Exp3_StartSize75_Wd0.025/EI_BS'+str(batch_size)+'_MSE_Ind' + str(ind) + ".npy"
filepath_ei_indices = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Results_v3/Exp3_StartSize75_Wd0.025/EI_BS"+str(batch_size)+"_SinglePt_Indices_Ind" + str(ind) + ".npy"
#exp_mse = []
exp_mse = list(np.load(filepath_ei_mse))

all_train_indices = np.load('/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Scripts_v2/trainindices'+ str(ind) + ".npy")
#train_data_indices = all_train_indices[0:size_train].tolist()
train_data_indices = list(np.load(filepath_ei_indices))
test_indices = np.load('/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Scripts_v2/testindices'+ str(ind) + ".npy")

X_train = X[train_data_indices]
y_train = y[train_data_indices]
X_test = X[test_indices]
y_test = y[test_indices]
print(X_train.shape)

#First active learning iteration
model = initialize_model(size_train)
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batchsize, verbose=0)
f =  K.function([model.layers[0].input, K.learning_phase()], 
               [model.layers[-1].output])
predictions_with_uncertainty = predict_with_uncertainty(f, X_test, mse_dropout_iterations)
y_predicted = predictions_with_uncertainty[0]
mse = np.mean(np.square(y_predicted-y_test))
#exp_mse.append(mse)

#Initializing pool indices
pool_indices = [ind for ind in all_train_indices if ind not in train_data_indices]
num_acquisitions = num_acquisitions - batch_size * (len(exp_mse) - 1)
print('Initial MSE: ' + str(exp_mse))

#Acquisition Loop
for acq in range(num_acquisitions//batch_size):
    #Randomly selecting sample points
    sample_indices = np.random.choice(len(pool_indices)+len(train_data_indices), int(sample_size*1.2), replace=False)
    pool_sample_ratio = len(sample_indices[sample_indices >= len(train_data_indices)])/sample_size
    train_sample_size = len(sample_indices[sample_indices < len(train_data_indices)])
    sample_indices_train = (sample_indices[sample_indices < len(train_data_indices)])[0:int(train_sample_size//pool_sample_ratio)]
    sample_indices_pool = (sample_indices[sample_indices >= len(train_data_indices)])[0:sample_size]
    X_train_fn = X[train_data_indices][sample_indices_train]
    X_pool_fn = np.concatenate((X[train_data_indices], X[pool_indices]))[sample_indices_pool]
    #Running EI acquisition
    acq_fn_results = get_acquisition_fn(model, X_train_fn, X_pool_fn, acq_fn_dropout_iterations, tau_inverse, batch_size, dropout_prob)
    acq_fn_ind = acq_fn_results
    acq_ind_ind = np.subtract(sample_indices_pool[acq_fn_ind], len(train_data_indices))
    acq_ind = np.array(pool_indices)[acq_ind_ind]
    #Labeling queried points
    for data_ind in acq_ind:
      train_data_indices.append(data_ind)
      pool_indices.remove(data_ind)    
    X_train = X[train_data_indices]
    y_train = y[train_data_indices]
    #Retraining EI model and calculating MSE
    model = initialize_model(len(train_data_indices))
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batchsize, verbose=0)
    f = K.function([model.layers[0].input, K.learning_phase()], 
               [model.layers[-1].output])
    predictions_with_uncertainty = predict_with_uncertainty(f, X_test, n_iter=mse_dropout_iterations)
    y_predicted = predictions_with_uncertainty[0]
    mse = np.mean(np.square(y_test-y_predicted))
    exp_mse.append(mse)
    print('Len Training Ind: ' + str(len(train_data_indices)) + ', last MSE: ' + str(exp_mse[-1]))
    if acq==(num_acquisitions//batch_size-1) or acq % 5 == 0:
      np.save(filepath_ei_indices, np.array(train_data_indices))
      np.save(filepath_ei_mse, np.array(exp_mse))


(511, 101, 4)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app



Initial MSE: [0.13188161020601427, 0.130107397798915, 0.13223755189365136, 0.14689247357106186, 0.1365815052149945, 0.14077400391977105, 0.1333263258373016, 0.14659042023639016, 0.1429058170853902, 0.13868524544768895, 0.12948372202206096, 0.14581154854530134, 0.13892168939890093, 0.13236213899908383, 0.12715244068507403, 0.1430938253329334, 0.13388317988664747, 0.13508209847050998, 0.13469591137453313, 0.13062269961101963, 0.13128188897743204, 0.12510363873829666, 0.1385827340388723, 0.12770044200449693, 0.13951059346222694, 0.12067209332383924, 0.12667180751597726, 0.12146358787386051, 0.13526510309990167, 0.13604838550339207, 0.11769880689937481, 0.12781058646244825, 0.12973170415769025, 0.12598293743457836, 0.11717500084392445, 0.12129692096242983, 0.12291529010210002, 0.12523503011804107, 0.12211446568659962, 0.11881061171411637, 0.11889339040352553, 0.12023617266905703, 0.12262832534443226, 0.12066188684744361, 0.1186604232707839, 0.11578710064876721, 0.11132968342025563, 0.1160

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app


Len Training Ind: 512, last MSE: 0.09440531703200694
num training: 18, num cand: 5000
Tau Inverse Value: 0.00025


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app


Len Training Ind: 700, last MSE: 0.08469730203414597
num training: 15, num cand: 5000
Tau Inverse Value: 0.00025


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app


Len Training Ind: 701, last MSE: 0.0890508798207752
num training: 22, num cand: 5000
Tau Inverse Value: 0.00025


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
